# Factor matching

This workbook is run using R

The purpose of this workbook is to investigate differnetially active groundtruth factors that were picked up with TL that are not picked up with Direct.

It is of interest to see the frequency with which differentially active groundtruth factors (which come from factorization of the reference datasets) are picked up as true positives from factorizations of the target datasets.
Then GSEA is used to assess the biological relevance of these factors, with speacial interest in those picked up with transfer learning but not with direct factorization.

## Identify factors of interest

In [ ]:
# libraries
library(MOFA2)
library(rhdf5)
library(rjson)
library(dplyr)
library(ggplot2)
library(RColorBrewer)
library(fgsea)
library(msigdbr)

In [2]:
# import the factorization evaluation list
### which configs
TopD = 5000
TrgFullTH = '01TH'
## import the scores
FctrznEvaluation = readRDS(file.path('Results',paste0('FctrznEvaluation_',TopD,'D_',TrgFullTH,'.rds')))
## extract record of matches
factor_matches = FctrznEvaluation[['factor_matches']]
## record number of iterations per project
fctrzn_subsets = FctrznEvaluation[['FctrznEvaluationResults']] %>% 
dplyr::group_by(prjct_name) %>%
dplyr::summarise(subsets = max(ss_iteration)) %>%
as.data.frame()

In [3]:
## loop through to aggregate
match_type = character()
prjct_name = character()
fctrzn_method = character()
fctr_number = integer()

for (m in names(factor_matches)){
    for (p in names(factor_matches[[m]])){
        for (ss in names(factor_matches[[m]][[p]])){
            for (fm in names(factor_matches[[m]][[p]][[ss]])){

                if (m=='True_positives'){
                   fctr_number_tmp = factor_matches[[m]][[p]][[ss]][[fm]]
                } else {
                    fctr_number_tmp = unique(factor_matches[[m]][[p]][[ss]][[fm]]$Full)
                }

                fctr_number = c(fctr_number, fctr_number_tmp) 
                fctrzn_method = c(fctrzn_method, rep(fm, length(fctr_number_tmp)))
                prjct_name = c(prjct_name, rep(p, length(fctr_number_tmp)))
                match_type = c(match_type, rep(m, length(fctr_number_tmp)))
                
            }
        }
    }
}

factor_matches_df = data.frame(
   match_type = match_type,
   prjct_name = prjct_name,
   fctrzn_method = fctrzn_method,
   fctr_number = fctr_number 
) %>% 
dplyr::inner_join(fctrzn_subsets, by = c('prjct_name')) %>%
dplyr::mutate(subsets_percent = 1/subsets) %>%
as.data.frame()



In [4]:
## tidy up and prepare summary table
factor_matches_df$fctrzn_method[factor_matches_df$fctrzn_method=="TL_VI"]="MOTL"

factor_matches_df$groundtruth_factor = factor(
    factor_matches_df$fctr_number, 
    levels = unique(factor_matches_df$fctr_number)[order(unique(factor_matches_df$fctr_number))]
    )

In [ ]:
# summary stats
PlotPrjcts = c('LAML_PAAD','LAML_SKCM','PAAD_SKCM','LAML_PAAD_SKCM')
PlotMatches = c('True_positives')
PlotName = 'MatchedFactors_TPs_'

PlotMethods = c('Direct', 'MOTL')

DFToPlot = factor_matches_df[
    is.element(factor_matches_df$prjct_name, PlotPrjcts) &
    is.element(factor_matches_df$match_type, PlotMatches) &
    is.element(factor_matches_df$fctrzn_method, PlotMethods) 
    ,]
DFToPlot$fctrzn_method = as.factor(DFToPlot$fctrzn_method)
DFToPlot$fctrzn_method = relevel(DFToPlot$fctrzn_method, ref = 'Direct')


ggplot(data = DFToPlot, mapping = aes(
    x=groundtruth_factor, fill=fctrzn_method, weight = subsets_percent
    )) +
    geom_bar(position = position_dodge2(preserve = "single")) +
    facet_grid(factor(prjct_name, levels = PlotPrjcts) ~ match_type, scales = 'free') +
    coord_flip()+
    xlab("Groundtruth factor") +
    ylab("Proportion of target datasets") +
    theme_bw() +
    theme(
        axis.text.x = element_text(size=6),
        axis.text.y = element_text(size=6),
        strip.text.x = element_text(size=6),
        strip.text.y = element_text(size=5),
        axis.title.x = element_text(size=6),
        axis.title.y = element_text(size=6),
        legend.title = element_blank(),
        legend.text = element_text(size=6)
        ) +
    scale_fill_brewer(palette = "Set2")

ggsave(file.path('Results',paste0(PlotName,TopD,'D_',TrgFullTH,'.pdf')),
         width = 10, height = 10, units = "cm"
         )
ggsave(file.path('Results',paste0(PlotName,TopD,'D_',TrgFullTH,'.png')),
         width = 10, height = 10, units = "cm"
         )

In [ ]:
# print a table
DFToTable = DFToPlot %>% 
dplyr::group_by(match_type, prjct_name, fctrzn_method, fctr_number) %>%
# dplyr::summarize(n = n()) %>%
dplyr::summarize(n = sum(subsets_percent)) %>%
    as.data.frame()

DFToTable = reshape(
    DFToTable,
    idvar = c("match_type", "prjct_name", "fctr_number"),
    timevar = "fctrzn_method",
    direction = "wide")


print(DFToTable[order(
    DFToTable$match_type,
    DFToTable$prjct_name, 
    DFToTable$fctr_number),])

write.table(DFToTable,
            file.path('Results', paste0(PlotName,TopD,'D_',TrgFullTH,'.csv')),
            quote = FALSE, sep=',', na='', row.names = FALSE, col.names = TRUE)


## Inspect a factor of interest

In [ ]:
## import a groundtruth factorization to inspect a factor
Prjct = 'LAML_PAAD'
TrgFullK = 100
TrgFullDir = file.path(
    paste0('Trg_',Prjct,'_Full_',TopD,'D'), 
    paste0('Fctrzn_',TrgFullK,'K_',TrgFullTH)
    )
InputModel = file.path(TrgFullDir,"Model.hdf5")
TrgFull_Fctrzn = load_model(file = InputModel)

print(colnames(TrgFull_Fctrzn@expectations$W[['mRNA']]))

In [ ]:
# plot variance explained by the factors and save plot
varexp_title = paste0(Prjct,' reference dataset')
varexp = plot_variance_explained(TrgFull_Fctrzn) + ggtitle(varexp_title)
varexp
ggsave(filename = file.path('Results',
                   paste0('VarExpl_',Prjct,'_',TopD,'D_',TrgFullK,'K_',TrgFullTH,'.pdf')),
                   plot = varexp
        #            ,
        #  width = 10, height = 10, units = "cm"
         )
ggsave(filename = file.path('Results',
                   paste0('VarExpl_',Prjct,'_',TopD,'D_',TrgFullK,'K_',TrgFullTH,'.png')),
                   plot = varexp
        #            ,
        #  width = 10, height = 10, units = "cm"
         )

In [ ]:
# print variance explained by the factors and save csv
varexp = TrgFull_Fctrzn@cache$variance_explained$r2_per_factor[[1]]
print(varexp)
# trick to export rownames to csv
colnames(varexp)[1] = paste0('Factor',',',colnames(varexp)[1])
write.table(varexp,
            file.path(
                'Results',
                paste0('VarExpl_',Prjct,'_',TopD,'D_',TrgFullK,'K_',TrgFullTH,'.csv')
                ),
            quote = FALSE, sep=',', na='', row.names = TRUE, col.names = TRUE)


In [ ]:
# Get the mRNA weight matrix
W_mRNA = TrgFull_Fctrzn@expectations$W[['mRNA']]
# truncate row names
rownames(W_mRNA) = substr(rownames(W_mRNA),1,regexpr('[.]',rownames(W_mRNA))-1)
# scale feature values across the factors?
ScaleW = function(x) {
  x/norm(matrix(x),type="F")
}
W_mRNA = t(apply(W_mRNA,1, ScaleW))

# pick a factor to use as the ranking vector
rankings = W_mRNA[,10]
names(rankings) = rownames(W_mRNA)
rankings = sort(rankings, decreasing = TRUE)
# check the rankings
max(rankings)
min(rankings)
plot(rankings)


In [ ]:
# what are the options for the gene sets?
msigdbr_species()
msigdbr_collections()

In [ ]:
## get a collection and create a list for fsgea
gene_sets = msigdbr(
    species = "Homo sapiens", 
    category = "C5", 
    subcategory = "GO:CC"
)

# colnames(gene_sets)
# print(head(gene_sets$ensembl_gene))

gene_sets = gene_sets %>% 
dplyr::distinct(gs_name, ensembl_gene) %>% 
as.data.frame()

## check output and overlap

overlap_genes = names(rankings)[is.element(names(rankings),gene_sets$ensembl_gene)]
length(overlap_genes)
print(head(gene_sets))

## create a list for fsgea

gene_sets = split(x = gene_sets$ensembl_gene, f = gene_sets$gs_name)
print(length(gene_sets))
head(names(gene_sets))

In [19]:
## run fsgea
fgseaRes = fgsea(
    pathways = gene_sets,
    stats = rankings,
    minSize = 15,
    maxSize = 500)

In [ ]:
## Quick inspection of GOCC_T_CELL_RECEPTOR_COMPLEX
fgseaRes$leadingEdge[fgseaRes$pathway=="GOCC_T_CELL_RECEPTOR_COMPLEX"]

In [ ]:
## check results
fgseaRes = fgseaRes %>% 
            dplyr::select(pathway, padj, ES, NES, size) %>% 
            as.data.frame()

fgseaRes = fgseaRes[fgseaRes$padj<0.01 & !is.na(fgseaRes$padj),]

print(head(fgseaRes))

## Loop to save significant pathways

The code before this is used to identify factors of interest and explore fgsea outputs.
This code should be run after restarting the kernel. It will loop through specified projects, factors and pathways and run fgsea. The final output is a dataframe to be saved as a csv

In [ ]:
# libraries
library(MOFA2)
library(rhdf5)
library(rjson)
library(dplyr)
library(ggplot2)
library(RColorBrewer)
library(fgsea)
library(msigdbr)

In [2]:
## common factorization parameter values
TopD = 5000
TrgFullTH = '01TH'
TrgFullK = 100

## factors to test 'LAML_PAAD','LAML_SKCM','PAAD_SKCM','LAML_PAAD_SKCM'
## Differentially active groundtruth factors picked up from T factorization
## that explained at least 0.01 of variance in R

FactorsToTest = list(
  LAML_PAAD = data.frame(
    k = c(1, 3),
    mRNAvar = c(75.5, 2.2),
    DirectFreq = c('middle','low')
  ),
  LAML_SKCM = data.frame(
    k = c(1, 3, 8, 9),
    mRNAvar = c(50.2, 3.9, 1.6, 3.0),
    DirectFreq = c('low','low','low', 'low')
  ),
  PAAD_SKCM = data.frame(
    k = c(2, 4, 7, 9, 10, 15),
    mRNAvar = c(35.6, 1.2, 5.1, 2.2, 2.1, 1.6),
    DirectFreq = c('low','high','low','low','low', 'low')
  ),
  LAML_PAAD_SKCM = data.frame(
    k = c(1, 2, 4, 5, 8),
    mRNAvar = c(26.7, 29.2, 3.9, 1.2, 2.6),
    DirectFreq = c('high','low','middle','low','low')
  )
)

## the genesets to test
GenesetsToTest = data.frame(
  category = c("C2", "C2", "C5", "C5"),
  subcategory = c("CP:KEGG", "CP:REACTOME", "GO:BP", "GO:CC")
)


In [ ]:
## empty list to store results
SigGeneSets = vector("list")

# scaling function
ScaleW = function(x) {
x/norm(matrix(x),type="F")
}

for (Prjct in names(FactorsToTest)){

    # import the groundtruth factorization
    TrgFullDir = file.path(
    paste0('Trg_',Prjct,'_Full_',TopD,'D'), 
    paste0('Fctrzn_',TrgFullK,'K_',TrgFullTH)
    )
    InputModel = file.path(TrgFullDir,"Model.hdf5")
    TrgFull_Fctrzn = load_model(file = InputModel) 

    # Get the mRNA weight matrix
    W_mRNA = TrgFull_Fctrzn@expectations$W[['mRNA']]
    # truncate row names
    rownames(W_mRNA) = substr(
        rownames(W_mRNA),
        1,
        regexpr('[.]',rownames(W_mRNA))-1
        )
    # scale the weights
    W_mRNA = t(apply(W_mRNA,1, ScaleW))

    for (ftt in 1:nrow(FactorsToTest[[Prjct]])){

        k = FactorsToTest[[Prjct]]$k[ftt]
        mRNAvar = FactorsToTest[[Prjct]]$mRNAvar[ftt]
        DirectFreq = FactorsToTest[[Prjct]]$DirectFreq[ftt]

        # pick a factor to use as the ranking vector
        rankings = W_mRNA[,k]
        names(rankings) = rownames(W_mRNA)

        # test for different geneset collections
        for (gstt in 1:nrow(GenesetsToTest)){
            # get the genesets for a selected collection
            category = GenesetsToTest$category[gstt]
            subcategory = GenesetsToTest$subcategory[gstt]
            gene_sets = msigdbr(
                species = "Homo sapiens", 
                category = category, 
                subcategory = subcategory
                )
                
            gene_sets = gene_sets %>% 
            dplyr::distinct(gs_name, ensembl_gene) %>% 
            as.data.frame()

            gene_sets = split(
                x = gene_sets$ensembl_gene, 
                f = gene_sets$gs_name
                )
            # set seed
            Seed = 1234567
            mode(Seed) = 'integer'
            set.seed(Seed)

            # run fgsea
            fgseaRes = fgsea(
                pathways = gene_sets,
                stats = rankings,
                minSize = 15,
                maxSize = 500
                )
            
            fgseaRes = fgseaRes %>% 
            dplyr::select(pathway, padj, ES, NES, size) %>% 
            as.data.frame()
            
            fgseaRes = fgseaRes[fgseaRes$padj<0.01 & !is.na(fgseaRes$padj),]
            
            fgseaRes$Prjct = rep(Prjct,nrow(fgseaRes))
            fgseaRes$k = rep(k,nrow(fgseaRes))
            fgseaRes$mRNAvar = rep(mRNAvar,nrow(fgseaRes))
            fgseaRes$DirectFreq = rep(DirectFreq,nrow(fgseaRes))
            fgseaRes$subcategory = rep(subcategory,nrow(fgseaRes))

            # add to list of results
            SigGeneSets[[paste0(c(Prjct,k,subcategory),collapse = ';')]] = fgseaRes
            # tidy up
            rm(list=c("fgseaRes", "gene_sets"))
            invisible(gc())

        } 
        # tidy up
        rm(list=c("rankings"))
        invisible(gc())              
    } 
    # tidy up
    rm(list=c("TrgFull_Fctrzn", "W_mRNA"))
    invisible(gc())   
}

## create a single dataframe
SigGeneSetsDF = do.call(rbind, SigGeneSets)
## export as csv
write.table(SigGeneSetsDF,
            file.path('Results','SigGeneSets.csv'),
            quote = FALSE, sep=',', na='', row.names = FALSE, col.names = TRUE)
